In [1]:
from coinbase.wallet.client import Client
import yaml
import os
import cbpro
import time
import pandas as pd

In [2]:
with open(os.path.expanduser("~/.ssh/go_trader_coinbase_key.yml"), "r") as stream:
    try:
        coinbase_tokens = yaml.safe_load(stream)["coinbase"]
    except yaml.YAMLError as exc:
        print(exc)

In [3]:
client = Client(api_key=coinbase_tokens["passphrase"], api_secret=coinbase_tokens["api_secret"])

### Test some public/private methods

In [5]:
# authenticate
auth_client = cbpro.AuthenticatedClient("gotrader", coinbase_tokens["api_secret"], coinbase_tokens["passphrase"])

- Get all fills

In [17]:
fills_gen = auth_client.get_fills("BTC-USD")
# Get all fills (will possibly make multiple HTTP requests)
all_fills = list(fills_gen)

In [18]:
all_fills

['message']

In [20]:
class myWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = "wss://ws-feed.pro.coinbase.com/"
        self.products = ["LTC-USD"]
        self.message_count = 0
        print("Lets count the messages!")

    def on_message(self, msg):
        self.message_count += 1
        if "price" in msg and "type" in msg:
            print("Message type:", msg["type"], "\t@ {:.3f}".format(float(msg["price"])))

    def on_close(self):
        print("-- Goodbye! --")


wsClient = myWebsocketClient()
wsClient.start()
print(wsClient.url, wsClient.products)
while wsClient.message_count < 500:
    print("\nmessage_count =", "{} \n".format(wsClient.message_count))
    time.sleep(1)
wsClient.close()

TypeError: __init__() missing 1 required keyword-only argument: 'channels'

In [21]:
import cbpro, time

order_book = cbpro.OrderBook(product_id="BTC-USD")
order_book.start()
time.sleep(10)
order_book.close()

-- Subscribed to OrderBook! --

Error: messages missing (29031085062 - 29031085159). Re-initializing  book at sequence.
Error: messages missing (29031085062 - 29031085160). Re-initializing  book at sequence.

-- OrderBook Socket Closed! --
-- OrderBook Socket Closed! --



In [6]:
# [ time, low, high, open, close, volume ]
# granularity = day
historic_rates = auth_client.get_product_historic_rates(product_id="BTC-USD", granularity=86400)

In [7]:
historic_rates[:10]

[[1631750400, 47361.01, 48217.33, 48144.03, 47486.65, 1837.63049455],
 [1631664000, 46705, 48475, 47127.81, 48148.12, 12139.84206628],
 [1631577600, 44679.92, 47274.92, 44953.23, 47127.22, 12644.16435358],
 [1631491200, 43465, 46900, 46027.8, 44947.72, 15015.30289712],
 [1631404800, 44754.31, 46462.98, 45173.66, 46024.23, 6499.85821749],
 [1631318400, 44730.29, 45989.94, 44850.37, 45171.83, 6207.38337004],
 [1631232000, 44140.48, 47040.76, 46396.26, 44851.45, 14905.54795341],
 [1631145600, 45511.82, 47400, 46059.94, 46400, 13594.64633403],
 [1631059200, 44423.49, 47381.47, 46894.49, 46060.36, 19624.55093117],
 [1630972800, 42830.77, 52944.96, 52698.8, 46894.5, 29022.815351]]

- verify this returns daily snapshots... It does

In [8]:
pd.to_datetime(historic_rates[0][0], unit="s")

Timestamp('2021-09-16 00:00:00')

In [46]:
pd.to_datetime(1630886400, unit="s")

Timestamp('2021-09-06 00:00:00')

In [47]:
pd.to_datetime(1630800000, unit="s")

Timestamp('2021-09-05 00:00:00')

In [49]:
pd.to_datetime(1630713600, unit="s")

Timestamp('2021-09-04 00:00:00')